In [0]:
ages_list = [21, 23, 18, 41, 32]

In [0]:
type(ages_list)

Out[2]: list

In [0]:
spark

Out[3]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
help(spark.createDataFrame)

Help on method createDataFrame in module pyspark.sql.session:

createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True) method of pyspark.sql.session.SparkSession instance
 Creates a :class:`DataFrame` from an :class:`RDD`, a list or a :class:`pandas.DataFrame`.
 
 When ``schema`` is a list of column names, the type of each column
 will be inferred from ``data``.
 
 When ``schema`` is ``None``, it will try to infer the schema (column names and types)
 from ``data``, which should be an RDD of either :class:`Row`,
 :class:`namedtuple`, or :class:`dict`.
 
 When ``schema`` is :class:`pyspark.sql.types.DataType` or a datatype string, it must match
 the real data, or an exception will be thrown at runtime. If the given schema is not
 :class:`pyspark.sql.types.StructType`, it will be wrapped into a
 :class:`pyspark.sql.types.StructType` as its only field, and the field name will be "value".
 Each record will also be wrapped into a tuple, which can be converted to row later.
 
 If schema inference is needed, ``samplingRatio`` is used to determined the ratio of
 rows used for schema inference. The first row will be used if ``samplingRatio`` is ``None``.
 
 .. versionadded:: 2.0.0
 
 .. versionchanged:: 2.1.0
 Added verifySchema.
 
 Parameters
 ----------
 data : :class:`RDD` or iterable
 an RDD of any kind of SQL data representation (:class:`Row`,
 :class:`tuple`, ``int``, ``boolean``, etc.), or :class:`list`, or
 :class:`pandas.DataFrame`.
 schema : :class:`pyspark.sql.types.DataType`, str or list, optional
 a :class:`pyspark.sql.types.DataType` or a datatype string or a list of
 column names, default is None. The data type string format equals to
 :class:`pyspark.sql.types.DataType.simpleString`, except that top level struct type can
 omit the ``struct<>`` and atomic types use ``typeName()`` as their format, e.g. use
 ``byte`` instead of ``tinyint`` for :class:`pyspark.sql.types.ByteType`.
 We can also use ``int`` as a short name for :class:`pyspark.sql.types.IntegerType`.
 samplingRatio : float, optional
 the sample ratio of rows used for inferring
 verifySchema : bool, optional
 verify data types of every row against schema. Enabled by default.
 
 Returns
 -------
 :class:`DataFrame`
 
 Notes
 -----
 Usage with spark.sql.execution.arrow.pyspark.enabled=True is experimental.
 
 Examples
 --------
 >>> l = [('Alice', 1)]
 >>> spark.createDataFrame(l).collect()
 [Row(_1='Alice', _2=1)]
 >>> spark.createDataFrame(l, ['name', 'age']).collect()
 [Row(name='Alice', age=1)]
 
 >>> d = [{'name': 'Alice', 'age': 1}]
 >>> spark.createDataFrame(d).collect()
 [Row(age=1, name='Alice')]
 
 >>> rdd = sc.parallelize(l)
 >>> spark.createDataFrame(rdd).collect()
 [Row(_1='Alice', _2=1)]
 >>> df = spark.createDataFrame(rdd, ['name', 'age'])
 >>> df.collect()
 [Row(name='Alice', age=1)]
 
 >>> from pyspark.sql import Row
 >>> Person = Row('name', 'age')
 >>> person = rdd.map(lambda r: Person(*r))
 >>> df2 = spark.createDataFrame(person)
 >>> df2.collect()
 [Row(name='Alice', age=1)]
 
 >>> from pyspark.sql.types import *
 >>> schema = StructType([
 ... StructField("name", StringType(), True),
 ... StructField("age", IntegerType(), True)])
 >>> df3 = spark.createDataFrame(rdd, schema)
 >>> df3.collect()
 [Row(name='Alice', age=1)]
 
 >>> spark.createDataFrame(df.toPandas()).collect() # doctest: +SKIP
 [Row(name='Alice', age=1)]
 >>> spark.createDataFrame(pandas.DataFrame([[1, 2]])).collect() # doctest: +SKIP
 [Row(0=1, 1=2)]
 
 >>> spark.createDataFrame(rdd, "a: string, b: int").collect()
 [Row(a='Alice', b=1)]
 >>> rdd = rdd.map(lambda row: row[1])
 >>> spark.createDataFrame(rdd, "int").collect()
 [Row(value=1)]
 >>> spark.createDataFrame(rdd, "boolean").collect() # doctest: +IGNORE_EXCEPTION_DETAIL
 Traceback (most recent call last):
 ...
 Py4JJavaError: ...

In [0]:
spark.createDataFrame(ages_list)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1173552103170700> in <module> 
 ----> 1 spark . createDataFrame ( ages_list ) 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 720 return super(SparkSession, self).createDataFrame(
 721 data, schema, samplingRatio, verifySchema)
 --> 722 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 
 723 
 724 def _create_dataframe ( self , data , schema , samplingRatio , verifySchema ) : 

 /databricks/spark/python/pyspark/sql/session.py in _create_dataframe (self, data, schema, samplingRatio, verifySchema) 
 752 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 753 else : 
 --> 754 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 755 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 756 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 528 write temp files . 
 529 """
 --> 530 data , schema = self . _wrap_data_schema ( data , schema ) 
 531 return self . _sc . parallelize ( data ) , schema
 532 

 /databricks/spark/python/pyspark/sql/session.py in _wrap_data_schema (self, data, schema) 
 507 
 508 if schema is None or isinstance ( schema , ( list , tuple ) ) : 
 --> 509 struct = self . _inferSchemaFromList ( data , names = schema ) 
 510 converter = _create_converter ( struct ) 
 511 data = map ( converter , data ) 

 /databricks/spark/python/pyspark/sql/session.py in _inferSchemaFromList (self, data, names) 
 438 if not data : 
 439 raise ValueError ( "can not infer schema from empty dataset" ) 
 --> 440 schema = reduce ( _merge_type , ( _infer_schema ( row , names ) for row in data ) ) 
 441 if _has_nulltype ( schema ) : 
 442 raise ValueError ( "Some of types cannot be determined after inferring" ) 

 /databricks/spark/python/pyspark/sql/session.py in <genexpr> (.0) 
 438 if not data : 
 439 raise ValueError ( "can not infer schema from empty dataset" ) 
 --> 440 schema = reduce ( _merge_type , ( _infer_schema ( row , names ) for row in data ) ) 
 441 if _has_nulltype ( schema ) : 
 442 raise ValueError ( "Some of types cannot be determined after inferring" ) 

 /databricks/spark/python/pyspark/sql/types.py in _infer_schema (row, names) 
 1065 
 1066 else : 
 -> 1067 raise TypeError ( "Can not infer schema for type: %s" % type ( row ) ) 
 1068 
 1069 fields = [ ] 

 TypeError : Can not infer schema for type: <class 'int'>

In [0]:
spark.createDataFrame(ages_list, 'int')

Out[6]: DataFrame[value: int]

In [0]:
from pyspark.sql.types import IntegerType

In [0]:
spark.createDataFrame(ages_list, IntegerType())

Out[8]: DataFrame[value: int]

In [0]:
names_list = ['Scott', 'Donald', 'Mickey']

In [0]:
spark.createDataFrame(names_list, 'string')

Out[13]: DataFrame[value: string]

In [0]:
from pyspark.sql.types import StringType

In [0]:
spark.createDataFrame(names_list, StringType())

Out[15]: DataFrame[value: string]

In [0]:
names_list

Out[18]: ['Scott', 'Donald', 'Mickey']

In [0]:
type(names_list)

Out[19]: list